In [1]:
import tskit, msprime
import numpy as np
import pandas as pd

In [2]:
def comparison_within(p1):
    num_sites = p1.shape[0]
    num_comparisons = p1.shape[1]
    result = []
    for i in range(num_comparisons):
        p2 = np.roll(p1,1, axis=1)
        result.append(np.sum(np.mod(p1+p2, 2))/num_sites)
    return result

In [3]:
def comparison_between(p1,p2):
    num_sites = p1.shape[0]
    num_comparisons = p1.shape[1]
    result = []
    for i in range(num_comparisons):
        result.append(np.sum(np.mod(p1+p2, 2))/num_sites)
        p2 = np.roll(p2,1, axis=1)
    return result

In [14]:
def Fst(ts):
    n = ts.sample_size//2
    G = ts.genotype_matrix()
    p1 = G[:,:n]
    p2 = G[:,n:]
    within = (np.mean(comparison_within(p1)) + np.mean(comparison_within(p2)))/2
    between = np.mean(comparison_between(p1, p2))
    total = np.mean([between, within])
    return (total - within)/total

In [15]:
#number of samples (2Xof diploid individuals) per deme:
sample_size = 4

#Rates for migration, recombination and mutation are unscaled and per generation:
mig = 3.8866e-7
seqLength = 32e3 
recr = 1.84675e-8
Ne0 = 2.3241e6
Ne1 = 9.8922e5 
T = 4.8580e6
mu = 1.9e-9

population_configurations = [
    msprime.PopulationConfiguration(sample_size=sample_size, initial_size=Ne0),
    msprime.PopulationConfiguration(sample_size=sample_size, initial_size=Ne1),
    ]
#demographic events are specified in the order they occur backwards in time:
# The MassMigration specifies the split between population 0 and 1: 
# backwards in time all lineages from the smaller population (1) are derived from pop (0)
# at time T 
demographic_events = [
    msprime.PopulationParametersChange(time=T, initial_size=Ne0, population_id=0),
    msprime.MassMigration(time=T, source=1, destination=0, proportion=1.0),
    msprime.MigrationRateChange(T,0)
    ]
# migration matrix specifies a matrix of pairwise migration rates (backwards in time).
#migration_matrix = [[0,0],
#                    [mig,0]]

In [18]:
num_reps = 100
replicates = msprime.simulate(
        num_replicates = num_reps,
        length = seqLength, 
        recombination_rate = recr,
        population_configurations = population_configurations,
        demographic_events = demographic_events,
        migration_matrix = [[0,0],[mig,0]],
        mutation_rate = mu)

In [19]:
fstsim = np.zeros(num_reps)
for index, ts in enumerate(replicates):
    fstsim[index] = Fst(ts)

In [20]:
np.mean(fstsim)

0.18193851575085584